In [1]:
import pandas as pd
import numpy as np
import altair as alt
from spacepy.coordinates import Coords
from spacepy.time import Ticktock
from datetime import datetime, timedelta
from pytz import timezone
import pytz
import os
alt.data_transformers.disable_max_rows()
os.makedirs("outputs/", exist_ok = True)
os.makedirs("geo_images/", exist_ok = True)

In [30]:
df = pd.read_csv("/cluster/home/rodrigm/Ionosphere_new/IonosphereModel/outputs/model_2025-05-23-13-49-44SA20241001.csv")
doy = 183
year = 2024
month = 7
day =  1

In [31]:
# Undo sine / cosine transformation on second
angle = (np.array(df["sod_sin"]) >= 0)*np.arccos(np.array(df["sod_cos"])) + (np.array(df["sod_sin"]) < 0)*(2*np.pi-np.arccos(np.array(df["sod_cos"])))
sod =  (86400*angle) / (2 * np.pi)
df["sod"] = sod
df = df[df["sod"] < 24*60*60]

# MAE

In [32]:
target = np.array(df["target"])
pred = np.array(df["prediction"])
print("MAE",np.mean(np.abs(target-pred)))

MAE 4.556109511571249


### Recover input

In [5]:
# Undo sine / cosine transformation on latitude
angle = (np.array(df["sm_lon_ipp_sin"]) >= 0)*np.arccos(np.array(df["sm_lon_ipp_cos"])) + (np.array(df["sm_lon_ipp_sin"]) < 0)*(2*np.pi-np.arccos(np.array(df["sm_lon_ipp_cos"])))
angle *= 360 / (2*np.pi)
lons = angle

In [6]:
lats = df["sm_lat_ipp"]

### Error by elevation angle

In [7]:
df.head(2)

,sm_lat_ipp,sm_lon_ipp_sin,sm_lon_ipp_cos,sod_sin,sod_cos,satazi_sin,satazi_cos,satele,doy,year,kp_index_daily,r_index_daily,dst_index_daily,f_index_daily,prediction,target,sod
0,-21.316343,0.860202,-0.509954,0.0,1.0,0.974851,0.222859,15.287000,100.0,2024.0,20.0,56.0,-7.0,124.800003,33.170132,21.573999,0.0
1,-24.254080,0.959104,-0.283055,0.0,1.0,-0.993298,-0.115578,50.733002,100.0,2024.0,20.0,56.0,-7.0,124.800003,19.306767,16.679001,0.0


In [8]:
ele_th_list = np.arange(5,90,5)
ele_list = np.array(df["satele"])
ele_error_list = []
ele_sample_size = []
for ele_start in np.unique(ele_th_list):
    ele_mask = np.logical_and(ele_list >= ele_start, ele_list < ele_start+0.1)
    ele_error_list.append(np.mean(np.abs(pred[ele_mask]-target[ele_mask])))
    ele_sample_size.append(np.sum(ele_mask))

In [9]:
avg_th = np.arange(7.5,90,5)
plot_df = pd.DataFrame()
plot_df["elevation angle"] = avg_th
plot_df["MAE"] = ele_error_list
alt.Chart(plot_df).mark_circle().encode(
    x="elevation angle",
    y="MAE"
)

alt.Chart(...)

In [10]:
plot_df["MAE"] = ele_error_list
alt.Chart(plot_df).mark_bar().encode(
    x="elevation angle",
    y="MAE"
)

alt.Chart(...)

### Convert back to GEO coordinates

In [11]:
# Converse solar magnetic coordinates to geo coordinates
def coord_transform(input_type, output_type, lats, lons, epochs):
    coords = np.array([[1 + 450 / 6371, lat, lon] for lat, lon in zip(lats, lons)], dtype=np.float64)
    geo_coords = Coords(coords, input_type, 'sph')
    geo_coords.ticks = Ticktock(epochs, 'UTC')
    return geo_coords.convert(output_type, 'sph')

date = datetime.strptime("2024-01-01", "%Y-%m-%d") + timedelta(days=doy - 1)
epochs = [date + timedelta(seconds=int(sod)) for sod in df["sod"]]
sm_coords = coord_transform('SM', 'GEO', lats, lons, epochs)
out_coords = sm_coords.data

### Get timezone / local hour / region

In [12]:
# Find area of prediction
from timezonefinder import TimezoneFinder
tf = TimezoneFinder()  # reuse
tz_list = []
query_points = zip(out_coords[:,1],out_coords[:,2])
for lat, lng in query_points:
    tz = tf.timezone_at(lng=lng, lat=lat)  # 'Europe/Berlin'
    tz_list.append(tz)
tz_list = np.array(tz_list)
df["timezone_name"] = tz_list

In [13]:
# Calculate local time
local_hour_list = []
for _ , row in df.iterrows():
    s = row["sod"]
    h = int(s/3600)
    global_time = datetime(year,month,day,h,0,0,tzinfo=pytz.utc)

    local_zone = row["timezone_name"]
    if local_zone == 'America/Coyhaique':
        local_zone = "America/Santiago"

    local_hour = global_time.astimezone(timezone(local_zone)).hour
    local_hour_list.append(local_hour)
    
df["local_hour"] = local_hour_list

In [14]:
np.min(df["sod"]), np.max(df["sod"])

(np.float64(0.0), np.float64(86369.97251173368))

In [15]:
region_list = np.array([s.split('/')[0] for s in tz_list])
df["region"] = region_list

### Error by region

In [16]:
region_error_list = []
region_sample_size = []
for current_region in np.unique(region_list):
    region_mask = current_region == region_list
    region_error_list.append(np.mean(np.abs(pred[region_mask]-target[region_mask])))
    region_sample_size.append(np.sum(region_mask))
    

In [28]:
plot_df = pd.DataFrame()
plot_df["region"] = np.unique(region_list)
plot_df["MAE"] = region_error_list
alt.Chart(plot_df).mark_bar().encode(
    x="region:N",
    y="MAE"
).configure_axis(labelFontSize=40,titleFontSize=50,labelAngle=0).properties(width=3000,height=3000).save("region_error_HD.png")

### Error by local time

In [18]:
local_time_error_list = []
local_time_sample_size = []
for lt in np.unique(local_hour_list):
    local_time_mask = local_hour_list == lt
    local_time_error_list.append(np.mean(np.abs(pred[local_time_mask]-target[local_time_mask])))
    local_time_sample_size.append(np.sum(local_time_mask))

In [19]:
plot_df = pd.DataFrame()
plot_df["local time"] = np.unique(local_hour_list)
plot_df["MAE"] = local_time_error_list
alt.Chart(plot_df).mark_bar().encode(
    x="local time",
    y="MAE"
).properties(width=500,height=500)

alt.Chart(...)